## We recommend running the Code in high GPU and RAM otherwise transformer model will show this warning---- 
CUDA out of memory. Tried to allocate 768.00 MiB (GPU 0; 15.90 GiB total capacity; 14.32 GiB already allocated; 455.75 MiB free; 14.35 GiB reserved in total by PyTorch) 

## Data Set Loading

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# directory
%cd /content/drive/MyDrive/Competition

/content/drive/MyDrive/Competition


In [ ]:
import numpy as np
y_train = np.load('y_train.npy')
#y_train.shape
#y_train

In [ ]:
import pandas as pd
df_label = pd.DataFrame(y_train)
df_label.to_csv('df_label.csv')

In [ ]:
!pip install datasets

     |████████████████████████████████| 325 kB 8.2 MB/s 
     |████████████████████████████████| 136 kB 42.2 MB/s 
     |████████████████████████████████| 212 kB 36.1 MB/s 
     |████████████████████████████████| 1.1 MB 60.7 MB/s 
     |████████████████████████████████| 77 kB 6.6 MB/s 
     |████████████████████████████████| 127 kB 44.1 MB/s 
     |████████████████████████████████| 144 kB 54.3 MB/s 
     |████████████████████████████████| 94 kB 3.0 MB/s 
     |████████████████████████████████| 271 kB 52.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
#np.savetxt('txt_y_train.csv', y_train)

In [ ]:
from datasets import load_dataset
dataset = load_dataset('text', data_files={'train': ['X_train.txt'], 'test': ['X_test.txt']})

Using custom data configuration default-ce94b226e1e35eca


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-ce94b226e1e35eca/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from datasets import load_dataset
dataset_label = load_dataset('text', data_files='hello.txt')

Using custom data configuration default-21b3a35d89ad7004


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-21b3a35d89ad7004/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset_label

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1000
    })
})

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['text'],
        num_rows: 792
    })
})

In [ ]:
dataset['train'][0]

{'text': 'Came here for the witches, stayed for the Gwent.'}

In [ ]:
pip install -q transformers

     |████████████████████████████████| 4.0 MB 8.1 MB/s 
     |████████████████████████████████| 6.6 MB 50.3 MB/s 
     |████████████████████████████████| 880 kB 51.8 MB/s 
     |████████████████████████████████| 596 kB 50.7 MB/s 


In [ ]:
from transformers import AutoTokenizer
model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
from transformers import AutoModelForSequenceClassification
num_labels = 2
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = (AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels).to(device))

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classi

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  f1 = f1_score(labels, preds, average='weighted')
  acc = accuracy_score(labels, preds)
  return {"accuracy": acc, "f1": f1}

In [ ]:
def tokenize(batch):
  return tokenizer(batch["text"], padding = True, truncation = True)

In [ ]:
emotion_encoded = dataset.map(tokenize, batched = True, batch_size=None)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
from transformers import Trainer, TrainingArguments
batch_size = 64
logging_steps = len(emotion_encoded["train"])//batch_size

In [ ]:
model_name = f"{model_ckpt}-finetuned-competition"

In [ ]:
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=2,
                                  learning_rate = 2e-5,
                                  per_device_train_batch_size = batch_size,
                                  per_device_eval_batch_size= batch_size,
                                  weight_decay = 0.01,
                                  evaluation_strategy = "epoch",
                                  disable_tqdm = False,
                                  logging_steps = logging_steps, 
                                  #push_to_hub = True,
                                  log_level="error")

In [ ]:
from transformers import Trainer

trainer = Trainer(model=model, args=training_args, 
                  compute_metrics = compute_metrics,
                  train_dataset=emotion_encoded["train"],
                  #eval_dataset=emotion_encoded["test"],
                  tokenizer=tokenizer)

trainer.train()

In [ ]:
class_names = ["positive", "negative"]
emotion_features = Features({'text': Value('string'), 'label': ClassLabel(names=class_names)})

In [ ]:
# making prediction
preds_output = trainer.predict(emotion_encoded["test"])
y_preds = np.argmax(preds_output.predictions, axis=1)
#Y_preds gives the predicted label of test data
print(y_preds)